In [30]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import pickle 
import os.path 
import base64 
import email 
from bs4 import BeautifulSoup
import csv

import gspread
from google.oauth2.service_account import Credentials

In [40]:
SCOPES = ["https://www.googleapis.com/auth/gmail.send",'https://www.googleapis.com/auth/gmail.readonly','https://mail.google.com/', "https://www.googleapis.com/auth/gmail.metadata", "https://www.googleapis.com/auth/gmail.modify"]
client_secret_file = './credenciales/client_secret_python_gmail.json'


In [88]:
def getEmails(): 
    # Variable creds will store the user access token. 
    # If no valid token found, we will create one. 
    creds = None
  
    # The file token.pickle contains the user access token. 
    # Check if it exists 
    if os.path.exists('token.pickle'): 
  
        # Read the token from the file and store it in the variable creds 
        with open('token.pickle', 'rb') as token: 
            creds = pickle.load(token) 
  
    # If credentials are not available or are invalid, ask the user to log in. 
    if not creds or not creds.valid: 
        if creds and creds.expired and creds.refresh_token: 
            creds.refresh(Request()) 
        else: 
            flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES) 
            creds = flow.run_local_server(port=0) 
  
        # Save the access token in token.pickle file for the next run 
        with open('token.pickle', 'wb') as token: 
            pickle.dump(creds, token) 
  
    # Connect to the Gmail API 
    service = build('gmail', 'v1', credentials=creds) 
  
    # request a list of all the messages 
    result = service.users().messages().list(userId='me').execute() 
  
    # We can also pass maxResults to get any number of emails. Like this: 
    # result = service.users().messages().list(maxResults=200, userId='me').execute() 
    messages = result.get('messages') 
  
    # messages is a list of dictionaries where each dictionary contains a message id. 
  
    # iterate through all the messages 
    for msg in messages: 
        # Get the message from its id 
        txt = service.users().messages().get(userId='me', id=msg['id']).execute() 
  
        # Use try-except to avoid any Errors 
        try: 
            # Get value of 'payload' from dictionary 'txt' 
            payload = txt['payload'] 
            headers = payload['headers'] 
  
            # Look for Subject and Sender Email in the headers 
            for d in headers: 
                if d['name'] == 'Subject': 
                    subject = d['value'] 
                if d['name'] == 'From': 
                    sender = d['value'] 
  
            # The Body of the message is in Encrypted format. So, we have to decode it. 
            # Get the data and decode it with base 64 decoder. 
            parts = payload.get('parts')[0] 
            data = parts['body']['data'] 
            data = data.replace("-","+").replace("_","/") 
            decoded_data = base64.b64decode(data) 
            
  
            # Now, the data obtained is in lxml. So, we will parse  
            # it with BeautifulSoup library 
            soup = BeautifulSoup(decoded_data , "csv") 
            body = soup.body() 
  
            # Printing the subject, sender's email and message 
            print("Subject: ", subject) 
            print("From: ", sender) 
            print("Message: ", body) 
            print('\n') 
        except: 
            pass
  
  
getEmails()

## conexion

In [89]:
creds = None
  
# The file token.pickle contains the user access token. 
# Check if it exists 
if os.path.exists('token.pickle'): 
  
    # Read the token from the file and store it in the variable creds 
    with open('token.pickle', 'rb') as token: 
        creds = pickle.load(token) 
  
# If credentials are not available or are invalid, ask the user to log in. 
if not creds or not creds.valid: 
    if creds and creds.expired and creds.refresh_token: 
        creds.refresh(Request()) 
    else: 
        flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES) 
        creds = flow.run_local_server(port=0) 
  
    # Save the access token in token.pickle file for the next run 
    with open('token.pickle', 'wb') as token: 
        pickle.dump(creds, token) 
  
# Connect to the Gmail API 
service = build('gmail', 'v1', credentials=creds) 
  
# request a list of all the messages 
result = service.users().messages().list(userId='me').execute() 
  
# We can also pass maxResults to get any number of emails. Like this: 
# result = service.users().messages().list(maxResults=200, userId='me').execute() 
messages = result.get('messages') 

## mensajes

In [13]:
messages = list_messages(service)
#for message in messages:
    #print(f"ID del mensaje: {message['id']}")

In [87]:
def get_message(service, user_id='me', msg_id=''):
    try:
        message = service.users().messages().get(userId=user_id, id=msg_id).execute()
        return message
    except Exception as error:
        print(f'Error al obtener el mensaje: {error}')

def get_attachment(service, user_id='me', msg_id='', store_dir=''):
    try:
        message = get_message(service, user_id, msg_id)

        for part in message['payload']['parts']:
            if part['filename']:
                file_data = base64.urlsafe_b64decode(part['body']['data'].encode('UTF-8'))

                # Guarda el archivo adjunto en el directorio especificado
                path = os.path.join(store_dir, part['filename'])
                with open(path, 'wb') as f:
                    f.write(file_data)
                    print(f'Archivo adjunto guardado en: {path}')

                # Procesa el archivo CSV después de guardarlo
                process_csv_file(path)

    except Exception as error:
        print(f'Error al obtener el adjunto: {error}')

def process_csv_file(csv_file_path):
    try:
        with open(csv_file_path, 'r') as csv_file:
            csv_reader = csv.reader(csv_file)
            for row in csv_reader:
                # Procesa cada fila del archivo CSV como desees
                print(row)
    except Exception as error:
        print(f'Error al procesar el archivo CSV: {error}')

In [86]:
# Lista los mensajes en la bandeja de entrada
messages = list_messages(service)

# Obtiene el primer mensaje (cambia el índice según tu necesidad)
if messages:
    first_message_id = messages[1]['id']
    get_attachment(service, msg_id=first_message_id, store_dir='./files')

Error al obtener el adjunto: 'data'


In [33]:
def search_emails_by_title(service, user_id, title):
    try:
        response = service.users().messages().list(userId=user_id, q=f'subject:{title}').execute()
        messages = response.get('messages', [])

        if not messages:
            print('No se encontraron correos electrónicos con ese título.')
        else:
            print('Correos electrónicos encontrados:')
            for message in messages:
                print(f'ID del mensaje: {message["id"]}')

    except Exception as e:
        print(f'Error al buscar correos electrónicos: {e}')

# Define el ID del usuario (generalmente es 'me' para el usuario autenticado)
user_id = 'me'

# Especifica el título del correo electrónico que estás buscando
email_title = 'test gmail csv'

# Llama a la función de búsqueda
search_emails_by_title(service, user_id, email_title)

Correos electrónicos encontrados:
ID del mensaje: 18c1ae8bcff0d28c


In [58]:
def download_attachment(service, user_id, title, download_path):
    try:
        response = service.users().messages().list(userId=user_id, q=f'subject:{title}').execute()
        messages = response.get('messages', [])

        if not messages:
            print('No se encontraron correos electrónicos con ese título.')
        else:
            print('Correos electrónicos encontrados:')
            for message in messages:
                print(message)
                message_id = message['id']
                msg = service.users().messages().get(userId=user_id, id=message_id).execute()

                # Obtiene el título del correo electrónico
                email_subject = msg['subject']
                print(f'Título del correo electrónico: {email_subject}')

                # Verifica si hay adjuntos
                if 'parts' in msg['payload']:
                    for part in msg['payload']['parts']:
                        if 'filename' in part:
                            filename = part['filename']
                            body = part['body']

                            # Verifica si el adjunto es un archivo CSV
                            if 'text/csv' in body['attachmentId']:
                                attachment = service.users().messages().attachments().get(
                                    userId=user_id, messageId=message_id, id=body['attachmentId']).execute()
                                print(attachment)

                                file_data = base64.urlsafe_b64decode(attachment['data'])
                                file_path = os.path.join(download_path, filename)

                                # Guarda el archivo CSV localmente
                                with open(file_path, 'wb') as f:
                                    f.write(file_data)

                                print(f'Adjunto descargado en: {file_path}')

    except Exception as e:
        print(f'Error al buscar y descargar adjuntos: {e}')

# Define el ID del usuario (generalmente es 'me' para el usuario autenticado)
user_id = 'me'

# Especifica el título del correo electrónico que estás buscando
email_title = 'test gmail csv'

# Especifica el directorio de descarga para el adjunto
download_directory = './files'

# Llama a la función de búsqueda y descarga de adjuntos
download_attachment(service, user_id, email_title, download_directory)

Correos electrónicos encontrados:
{'id': '18c1b2b9b9257cb3', 'threadId': '18c1b2b2ef01d93d'}
Error al buscar y descargar adjuntos: 'subject'


In [63]:
service = build('gmail', 'v1', credentials=creds) 
  
    # request a list of all the messages 
result = service.users().messages().list(userId='me').execute() 
  
    # We can also pass maxResults to get any number of emails. Like this: 
    # result = service.users().messages().list(maxResults=200, userId='me').execute() 
messages = result.get('messages')

In [68]:
for msg in messages: 
    # Get the message from its id 
    txt = service.users().messages().get(userId='me', id=msg['id']).execute()

In [69]:
txt

{'id': '18ba3e2bbcd893c0',
 'threadId': '18ba3e2bbcd893c0',
 'labelIds': ['IMPORTANT', 'CATEGORY_UPDATES', 'INBOX'],
 'snippet': 'Ajustes de notificaciones MIDE - Looker Studio Octubre 2023 Nuevo anuncio Estimado alumno. Te adjuntamos las indicaciones para incluir dentro de tu perfil de Linkedin, el certificado que acredita tu',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To', 'value': 'alvarosaezsanchez@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a05:7010:934b:b0:393:73cd:86f8 with SMTP id s11csp2486296mdk;        Mon, 6 Nov 2023 01:07:07 -0800 (PST)'},
   {'name': 'X-Received',
    'value': 'by 2002:a05:6e02:b4d:b0:351:ab2:2621 with SMTP id f13-20020a056e020b4d00b003510ab22621mr41350541ilu.15.1699261626953;        Mon, 06 Nov 2023 01:07:06 -0800 (PST)'},
   {'name': 'ARC-Seal',
    'value': 'i=1; a=rsa-sha256; t=1699261626; cv=none;        d=google.com; s=arc-20160816;        b=zTmtUU7T9VLRJe6V75

------------------------

In [73]:
response2 = service.users().messages().list(userId='me', q=f'subject:test gmail csv').execute()
messages2 = response2.get('messages')

In [74]:
messages2

[{'id': '18c1b2b9b9257cb3', 'threadId': '18c1b2b2ef01d93d'},
 {'id': '18c1ae8bcff0d28c', 'threadId': '18c1ae886f268167'}]

In [75]:
for msg in messages2: 
    # Get the message from its id 
    txt = service.users().messages().get(userId='me', id=msg['id']).execute()

In [76]:
txt

{'id': '18c1ae8bcff0d28c',
 'threadId': '18c1ae886f268167',
 'labelIds': ['IMPORTANT', 'SENT', 'INBOX'],
 'snippet': '',
 'payload': {'partId': '',
  'mimeType': 'multipart/mixed',
  'filename': '',
  'headers': [{'name': 'MIME-Version', 'value': '1.0'},
   {'name': 'Date', 'value': 'Wed, 29 Nov 2023 12:47:52 +0100'},
   {'name': 'Message-ID',
    'value': '<CANFxmwZG6NJu-D6Jg1yEBD_SBNWF2XZ081ueMhHbcAz7M7JskQ@mail.gmail.com>'},
   {'name': 'Subject', 'value': 'test gmail csv'},
   {'name': 'From', 'value': 'Alv Sz <alvarosaezsanchez@gmail.com>'},
   {'name': 'To', 'value': 'Alv Sz <alvarosaezsanchez@gmail.com>'},
   {'name': 'Content-Type',
    'value': 'multipart/mixed; boundary="0000000000001d3aac060b491d55"'}],
  'body': {'size': 0},
  'parts': [{'partId': '0',
    'mimeType': 'multipart/alternative',
    'filename': '',
    'headers': [{'name': 'Content-Type',
      'value': 'multipart/alternative; boundary="0000000000001d3aaa060b491d53"'}],
    'body': {'size': 0},
    'parts': [{